In [1]:
%pip install pandas
%pip install python-dotenv
%pip install -U scikit-learn
%pip install sun_position_calculator
%pip install requests

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:

from typing import Dict


from models.econometrics import Cost, Currency

import math
from typing import Callable
from models.components import Component
from models.econometrics import Cost
from models.geometry import Orientation
from models.photovoltaic import Photovoltaic

Currency.CLP exchange ratios load status: True


In [3]:
from dotenv import dotenv_values
config = dotenv_values(".env.local")

# Building references

## Currencies values

In [4]:
import requests
import json
#currency values

# national units https://mindicador.cl/api
query_factor:requests.Response = requests.get('https://mindicador.cl/api',timeout=1000)
ratios_cl = json.loads(query_factor.text)
usd_clp:float = ratios_cl['dolar']['valor']
Cost.set_exchange(Currency.CLP,usd_clp)
Cost.set_exchange(Currency.UF,usd_clp/ratios_cl['uf']['valor']) # 1 dolar in UF
Cost.set_exchange(Currency.UTM,usd_clp/ratios_cl['utm']['valor']) # 1 dolar in Utm

# exchange rates https://app.freecurrencyapi.com/dashboard
query_exchange:requests.Response = requests.get(config["CURRENCY_API_KEY"],timeout=1000)

currency_ratios = json.loads(query_exchange.text)
Cost.set_exchange(Currency.EUR,currency_ratios['data']['EUR'])
Cost.set_exchange(Currency.GBP,currency_ratios['data']['GBP'])
Cost.set_exchange(Currency.BRL,currency_ratios['data']['BRL'])


set $1 Peso Chileno... on USD$0.00103 
set $1 UF............. on USD$38.42002 
set $1 UTM............ on USD$67.30618 
set $1 Euro........... on USD$1.06558 
set $1 Pound.......... on USD$1.23698 
set $1 Real........... on USD$0.19200 


## Address and general info🌎

In [5]:
#cspell:disable


#init proyect
from models.components import Tech
from models.geometry import GeoPosition
from models.inventory import Building, Project

project:Project = Project(
    title='Sistema Fotovoltaico OnGrid-Mixto',
    building = Building(
        geolocation=GeoPosition(latitude=-33.861702,longitude=-71.657839),
        name='Posta de Salud Rural Bucalemu',
        address='Ruta G-80-I km 30 sn, Localidad Bucalemu,\nSanto Domingo',
        city='Santo Domingo'),
    technology= [Tech.PHOTOVOLTAIC]
    )

# psr_sanenrique:Project = Project(
#     title='Sistema Fotovoltaico OnGrid-Mixto PSR San Enrique, Santo Domingo',
#     building = Building(
#         geolocation=GeoPosition(latitude=-33.9081421,longitude=-71.7159192),
#         name='Posta de Salud Rural de San Enrique',
#         address='Ruta G-866 km 0 sn, Localidad San Enrique, Santo Domingo',
#         city='Santo Domingo'),
#     technology= [Tech.PHOTOVOLTAIC]
#     )



api request URL https://power.larc.nasa.gov/api/temporal/hourly/point?Time=LST&parameters=T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M&community=RE&latitude=-33.861702&longitude=-71.657839&start=20230101&end=20230102&format=JSON


In [6]:
project.weather.get_data().head(24)

,date,T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M,year,month,day,hour
0,2022-12-31 19:00:00,15.99,0.00,0.00,None,None,1.61,2022,12,31,19
1,2022-12-31 20:00:00,15.62,0.00,0.00,None,None,1.69,2022,12,31,20
2,2022-12-31 21:00:00,15.29,0.00,0.00,None,None,1.91,2022,12,31,21
3,2022-12-31 22:00:00,14.89,0.00,0.00,None,None,2.08,2022,12,31,22
4,2022-12-31 23:00:00,14.46,0.00,0.00,None,None,2.37,2022,12,31,23
5,2023-01-01 00:00:00,14.20,0.00,0.00,None,None,2.73,2023,01,01,00
6,2023-01-01 01:00:00,13.96,0.00,0.00,None,None,2.96,2023,01,01,01
7,2023-01-01 02:00:00,13.66,0.00,0.00,None,None,2.99,2023,01,01,02
8,2023-01-01 03:00:00,13.34,0.00,0.00,None,None,3.09,2023,01,01,03
9,2023-01-01 04:00:00,13.16,0.00,0.00,None,None,3.15,2023,01,01,04


# Integration 

## Energy Consumptions🔌

In [7]:
from models.consumption import ElectricityBill, Energetic
project.building.add_consumptions(description="main",energetic=Energetic.ELI,consumption=[
    ElectricityBill(450,"30-12-2022",cost=71400),
    ElectricityBill(497,"30-01-2023",cost=80800),
    ElectricityBill(450,"28-02-2023",cost=71400),
    ElectricityBill(475,"30-03-2023",cost=77400),
    ElectricityBill(475,"30-04-2023",cost=77400),
    ElectricityBill(520,"30-05-2023",cost=93400),
    ElectricityBill(654,"30-06-2023",cost=122100),
    ElectricityBill(820,"30-07-2023",cost=140800),
    ElectricityBill(549,"31-10-2023",cost=100000),
])

Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True


In [8]:
project.building.consumptions['main'].forecast()

boundaries in month 8 : 820 <-> 549
boundaries in month 9 : 684.5 <-> 549
boundaries in month 11 : 549 <-> 450


,month,energy
0,1,497.00
1,2,450.00
2,3,475.00
3,4,475.00
4,5,520.00
5,6,654.00
6,7,820.00
7,8,684.50
8,9,616.75
9,10,549.00


## Machine Components 🎡

In [9]:
#cspell: disable

#PV COST MODEL, inscludes price clp per watt, including cost of planel, inverter, instalation and infraestructure.
pv_log_model_on_grid:Callable[[float],float] = lambda size_w: math.floor(-152.6*math.log(size_w)+2605.9) # clp/w
pv_log_model_off_grid:Callable[[float],float] = lambda size_w: math.floor(-414.7*math.log(size_w)+6143.9) # clp/w
#PV linear COST MODEL, inscludes price clp per watt,just panel.
pv_linear:Callable[[float],float] = lambda size_w: 245990/655

#clean previous load
project.components = {}
from IPython.display import clear_output
clear_output(wait=True)

# assembly projects components
project.add_component(
    'generación',
    Photovoltaic(project.weather,'Módulo Primario A',
                reference='https://www.tiendatecnored.cl/modulo-fotovoltaico-650w-canadian-solar.html',
                power=655,
                quantity=5,
                cost=None,
                cost_model=pv_linear,
                orientation=Orientation(15,270-16.46)),
    Photovoltaic(project.weather,'Módulo Secundario B',
                reference='https://www.tiendatecnored.cl/modulo-fotovoltaico-650w-canadian-solar.html',
                power=655,
                quantity=5,
                cost=None,
                cost_model=pv_linear,
                orientation=Orientation(15,16.46))
    )

project.add_component(
    'auxiliar',
    Component('inversor de 7kW',
                model='Voltronic 7kW 48V',
                reference='https://solarbex.com/comprar/inversor-hibrido-7kw-48v-voltronic/',
                specification='Híbrido 7kW Monofásico',
                cost_per_unit=Cost(1255,Currency.EUR)),
    Component('Controlador de Carga',
                model='Enertik 80A 12/24V',
                specification='1000W to 2000 24V output',
                cost_per_unit=Cost(101600,Currency.CLP)),
    Component('medidor bidireccional',
                model='Elster AS1440',
                specification='Trifasico',
                cost_per_unit=Cost(306990,Currency.CLP)),
    Component('almacenamiento',
                model='MP NP12-200',
                specification='Betería 12V 200Ah',
                reference='https://www.tiendatecnored.cl/bateria-gel-ciclo-profundo-12v-200ah.html',
                cost_per_unit=Cost(204889,Currency.CLP),
                quantity=10,
                )
    )

project.add_component(
    'obras',
    Component(description='instalación eléctrica',
            specification='H/P eléctrico',
            cost_per_unit=Cost(1,currency=Currency.UF),
            quantity=100,
            ),
    Component(description='estructura de soporte',
            specification='elementos fijación',
            cost_per_unit=Cost((4*800+11400/4+4*700+4000+500)),
            quantity=24
            )
)

Currency.CLP exchange ratios load status: True
inside cost pv :  245990.0 Currency.CLP
Currency.CLP exchange ratios load status: True
inside cost pv :  245990.0 Currency.CLP
Currency.EUR exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.UF exchange ratios load status: True
Currency.CLP exchange ratios load status: True


In [10]:
# psr_bucalemu.components['generación'][1].nominal_power()
#pv_log_model(1500) #clp/w

bucket_list = project.bucket_list(Currency.CLP)
    
#unitary price comparison
print(
    'estimado',bucket_list['cost'],
    'on grid',pv_log_model_on_grid(655*10)*655*10,
    'off grid',pv_log_model_off_grid(655*10)*655*10,
    'linear',pv_linear(655*10)*655*10)

Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.EUR exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.CLP exchange ratios load status: True
Currency.UF exchange ratios load status: True
Currency.CLP exchange ratios load status: True
estimado 11249048.55 on grid 8279200 off grid 16368450 linear 2459900.0


In [11]:
# psr_bucalemu.get_energy_generation('generación')
project.get_energy('generación')[['month','System_capacity_KW']].groupby('month',as_index=False).sum()

d:\Dev\energy-report-modeler\models\photovoltaic.py:189: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  irr.fillna(0.0,inplace=True)
d:\Dev\energy-report-modeler\models\photovoltaic.py:189: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  irr.fillna(0.0,inplace=True)


,month,System_capacity_KW
0,01,12.407308
1,12,0.000000


In [12]:
from models.emission import emission

emission.annual_projection(2024)
emission.annual_avg()


,year,emission
0,2018,0.418742
1,2019,0.405575
2,2020,0.383400
3,2021,0.390708
4,2022,0.300608
5,2023,0.242117


# Write Document 📒


In [13]:
from docxtpl import * 


ModuleNotFoundError: No module named 'docxtpl'

In [ ]:
#assembly proxy object 
ctx:dict[str,any] = {
    #about this project
    "project_name": project.title,
    "project_type" : project.technology[0],
    "project_size":"*test",
    "size_unit":"kW",
    "total_cost": project.bucket_list(Currency.CLP)["cost"],
    #site
    "building_name" : project.building.name,
    "city": project.building.city
}

In [ ]:
from docxtpl import DocxTemplate
doc = DocxTemplate("templates/memory_template.docx")
doc.render(ctx)
doc.save(f"build/reporte_{project.building.city}_{project.building.name}.docx")

project.get_context('generación')